In [22]:
from Bio import SeqIO
from tqdm import tqdm, tqdm_notebook
import numpy as np

from pathlib import Path

import pandas as pd
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from google_drive_downloader import GoogleDriveDownloader as gdd
from sklearn.feature_extraction.text import CountVectorizer
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm, tqdm_notebook

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
random_state = 4
device

device(type='cuda')

In [2]:
sg_reads_seq = []
sg_reads_tax = []

for seq_record in SeqIO.parse('data/16S-SG-reads.fa/16S-reads.fa', "fasta"):
    sg_reads_seq.append(seq_record.seq)
    sg_reads_tax.append(seq_record.description[seq_record.description.find('description=')+12+1:-1].split())
    
len(sg_reads_seq), len(sg_reads_tax)

(28224, 28224)

In [3]:
type(sg_reads_seq[0])

Bio.Seq.Seq

In [4]:
print(sg_reads_seq[0])

GCCCCGGGCTCAACCTGGGAACTGCATTTCGAACTGGCAAACTAGAGTTCTTGAGAGGGTGGTAGAATTTCAGGTGTAGCGGTGAAATGCGAAGAGATCTGAAGGATTACCAGTGGCGAAGGCGGCCTACCGGCAACTATCCGACTCTCGGATGCGAAAGCGAGGTACCGGACGGGATCCGTTGAGCTGTATTTGCCCGCGGTAAATGGATTCCGCTAGCGGGACTCGGAGAGCCTTAGTGGAACCAC


In [5]:
print(sg_reads_tax[0])

['Gammaproteobacteria', 'Alteromonadales', 'Moritellaceae', 'Moritella']


In [7]:
tax = ["Class", "Order", "Family", "Genus"]

In [8]:
sg = pd.DataFrame()
for i in range(4):
    col = []
    for j in sg_reads_tax:
        if (len(j))==3:
            j.append('Simiduia')
        col.append(j[i])
    sg[tax[i]] = col

In [9]:
sg.head()

,Class,Order,Family,Genus
0,Gammaproteobacteria,Alteromonadales,Moritellaceae,Moritella
1,Alphaproteobacteria,Rhodobacterales,Rhodobacteraceae,Sagittula
2,Gammaproteobacteria,"\""Vibrionales\""",Vibrionaceae,Lucibacterium
3,Alphaproteobacteria,Rhodobacterales,Rhodobacteraceae,Rhodobaca
4,Gammaproteobacteria,Alteromonadales,Ferrimonadaceae,Ferrimonas


# Word Vectors using  Bag of Words model

In [105]:
human = pd.DataFrame()
human['sequence'] = [str(i) for i in sg_reads_seq]

In [106]:
human.head()

,sequence
0,GCCCCGGGCTCAACCTGGGAACTGCATTTCGAACTGGCAAACTAGA...
1,ACTGCCTCCAAAACTATCAGTCTAGAGTTCGAGAGAGGTGAGTGGA...
2,TAGCGGCGTCGTTTGACGTTAGCGACAGAAGAAGCACCGGCTAACT...
3,CTAGCGTTGTTCGGAATTACTGGGCGTATAGCGCGCGTTAGGCGGA...
4,CATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACACATGCAAGTC...


In [107]:
# function to convert sequence strings into k-mer words, default size = 6 (hexamer words)
def getKmers(sequence, size=6):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

In [108]:
human['words'] = human.apply(lambda x: getKmers(x['sequence'], size=6), axis=1) # default size = 6
human = human.drop('sequence', axis=1)

In [109]:
human.head()

,words
0,"[gccccg, ccccgg, cccggg, ccgggc, cgggct, gggct..."
1,"[actgcc, ctgcct, tgcctc, gcctcc, cctcca, ctcca..."
2,"[tagcgg, agcggc, gcggcg, cggcgt, ggcgtc, gcgtc..."
3,"[ctagcg, tagcgt, agcgtt, gcgttg, cgttgt, gttgt..."
4,"[catggc, atggct, tggctc, ggctca, gctcag, ctcag..."


In [110]:
human_texts = list(human['words'])
for item in range(len(human_texts)):
    human_texts[item] = ' '.join(human_texts[item])
print(human_texts[0])

gccccg ccccgg cccggg ccgggc cgggct gggctc ggctca gctcaa ctcaac tcaacc caacct aacctg acctgg cctggg ctggga tgggaa gggaac ggaact gaactg aactgc actgca ctgcat tgcatt gcattt catttc atttcg tttcga ttcgaa tcgaac cgaact gaactg aactgg actggc ctggca tggcaa ggcaaa gcaaac caaact aaacta aactag actaga ctagag tagagt agagtt gagttc agttct gttctt ttcttg tcttga cttgag ttgaga tgagag gagagg agaggg gagggt agggtg gggtgg ggtggt gtggta tggtag ggtaga gtagaa tagaat agaatt gaattt aatttc atttca tttcag ttcagg tcaggt caggtg aggtgt ggtgta gtgtag tgtagc gtagcg tagcgg agcggt gcggtg cggtga ggtgaa gtgaaa tgaaat gaaatg aaatgc aatgcg atgcga tgcgaa gcgaag cgaaga gaagag aagaga agagat gagatc agatct gatctg atctga tctgaa ctgaag tgaagg gaagga aaggat aggatt ggatta gattac attacc ttacca taccag accagt ccagtg cagtgg agtggc gtggcg tggcga ggcgaa gcgaag cgaagg gaaggc aaggcg aggcgg ggcggc gcggcc cggcct ggccta gcctac cctacc ctaccg taccgg accggc ccggca cggcaa ggcaac gcaact caacta aactat actatc ctatcc tatccg atccga tccgac ccgact cgactc gactct

In [111]:
# Creating the Bag of Words model using CountVectorizer()
# This is equivalent to k-mer counting
# The n-gram size of 4 was previously determined by testing
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(4,4)) # initial : (4, 4)
X = cv.fit_transform(human_texts)

In [112]:
X.shape

(28224, 272193)

In [113]:
cv.vocabulary_

{'gccccg ccccgg cccggg ccgggc': 157785,
 'ccccgg cccggg ccgggc cgggct': 90258,
 'cccggg ccgggc cgggct gggctc': 91289,
 'ccgggc cgggct gggctc ggctca': 95447,
 'cgggct gggctc ggctca gctcaa': 112143,
 'gggctc ggctca gctcaa ctcaac': 179495,
 'ggctca gctcaa ctcaac tcaacc': 176760,
 'gctcaa ctcaac tcaacc caacct': 165902,
 'ctcaac tcaacc caacct aacctg': 122578,
 'tcaacc caacct aacctg acctgg': 221661,
 'caacct aacctg acctgg cctggg': 69159,
 'aacctg acctgg cctggg ctggga': 6164,
 'acctgg cctggg ctggga tgggaa': 24805,
 'cctggg ctggga tgggaa gggaac': 99683,
 'ctggga tgggaa gggaac ggaact': 129250,
 'tgggaa gggaac ggaact gaactg': 248516,
 'gggaac ggaact gaactg aactgc': 177685,
 'ggaact gaactg aactgc actgca': 169578,
 'gaactg aactgc actgca ctgcat': 137176,
 'aactgc actgca ctgcat tgcatt': 7922,
 'actgca ctgcat tgcatt gcattt': 31847,
 'ctgcat tgcatt gcattt catttc': 127933,
 'tgcatt gcattt catttc atttcg': 243216,
 'gcattt catttc atttcg tttcga': 156273,
 'catttc atttcg tttcga ttcgaa': 84217,
 'atttcg ttt

# Class 

In [24]:
print(list(sg.groupby('Class').count().index), len(sg.groupby('Class').count().index))

['Alphaproteobacteria', 'Betaproteobacteria', 'Gammaproteobacteria'] 3


In [25]:
classes = list(pd.get_dummies(sg['Class']).columns)
classes = ['class_' + i for i in classes]
print(classes)

['class_Alphaproteobacteria', 'class_Betaproteobacteria', 'class_Gammaproteobacteria']


In [26]:
labels_classes = np.argmax(np.array(pd.get_dummies(sg['Class'])), axis=1)
print(labels_classes, len(labels_classes))

[2 0 2 ... 0 0 2] 28224


In [27]:
# Splitting the human dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, 
                                                    labels_classes, 
                                                    test_size = 0.10, 
                                                    random_state=42)

In [30]:
print(X_train_c.shape)
print(X_test_c.shape)

(25401, 272193)
(2823, 272193)


In [31]:
### Multinomial Naive Bayes Classifier ###
# The alpha parameter was determined by grid search previously
from sklearn.naive_bayes import MultinomialNB
classifier_c = MultinomialNB(alpha=0.1)
classifier_c.fit(X_train_c, y_train_c)

MultinomialNB(alpha=0.1)

In [32]:
y_pred_c = classifier_c.predict(X_test_c)
y_pred_c

array([2, 0, 0, ..., 0, 2, 0], dtype=int64)

In [33]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
print("Confusion matrix\n")
print(pd.crosstab(pd.Series(y_test_c, name='Actual'), pd.Series(y_pred_c, name='Predicted')))
def get_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1 = get_metrics(y_test_c, y_pred_c)
print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))

Confusion matrix

Predicted     0    1     2
Actual                    
0          1301    0     2
1             3  191     1
2             3    1  1321
accuracy = 0.996 
precision = 0.996 
recall = 0.996 
f1 = 0.996


In [34]:
def predict_c(seq):
    if type(seq)== str:
        seq = [seq]
    return classifier_c.predict(cv.transform(seq))
def find_class(seq):
    pred = predict_c(seq)
    return [classes[i] for i in pred]

In [35]:
predict_c(human_texts[0]).item()

2

In [36]:
find_class(human_texts[0:5]), find_class(human_texts[0])

(['class_Gammaproteobacteria',
  'class_Alphaproteobacteria',
  'class_Gammaproteobacteria',
  'class_Alphaproteobacteria',
  'class_Gammaproteobacteria'],
 ['class_Gammaproteobacteria'])

# Order

In [37]:
print(list(sg.groupby('Order').count().index), len(sg.groupby('Order').count().index))

['Aeromonadales', 'Alteromonadales', 'Caulobacterales', 'Chromatiales', 'Ferrovales', 'Gammaproteobacteria_incertae_sedis', 'Legionellales', 'Methylophilales', 'Neisseriales', 'Oceanospirillales', 'Pasteurellales', 'Pseudomonadales', 'Rhizobiales', 'Rhodobacterales', 'Rhodospirillales', 'Sphingomonadales', 'Thiotrichales', 'Vibrionales', 'Xanthomonadales', '\\"Salinisphaerales\\"', '\\"Vibrionales\\"'] 21


In [38]:
orders = []
for i in sg['Order']:
    if i[0:2]==   r"\"":
        i = i[2:-2]
    orders.append(i)

In [39]:
sg['Order'] = orders

In [40]:
print(list(sg.groupby('Order').count().index), len(sg.groupby('Order').count().index))

['Aeromonadales', 'Alteromonadales', 'Caulobacterales', 'Chromatiales', 'Ferrovales', 'Gammaproteobacteria_incertae_sedis', 'Legionellales', 'Methylophilales', 'Neisseriales', 'Oceanospirillales', 'Pasteurellales', 'Pseudomonadales', 'Rhizobiales', 'Rhodobacterales', 'Rhodospirillales', 'Salinisphaerales', 'Sphingomonadales', 'Thiotrichales', 'Vibrionales', 'Xanthomonadales'] 20


In [45]:
orders = list(pd.get_dummies(sg['Order']).columns)
orders = ['order_' + i for i in orders]

labels_orders = np.argmax(np.array(pd.get_dummies(sg['Order'])), axis=1)
print(labels_orders, len(labels_orders))

[ 1 13 18 ... 13 12 19] 28224


In [46]:
print(orders)

['order_Aeromonadales', 'order_Alteromonadales', 'order_Caulobacterales', 'order_Chromatiales', 'order_Ferrovales', 'order_Gammaproteobacteria_incertae_sedis', 'order_Legionellales', 'order_Methylophilales', 'order_Neisseriales', 'order_Oceanospirillales', 'order_Pasteurellales', 'order_Pseudomonadales', 'order_Rhizobiales', 'order_Rhodobacterales', 'order_Rhodospirillales', 'order_Salinisphaerales', 'order_Sphingomonadales', 'order_Thiotrichales', 'order_Vibrionales', 'order_Xanthomonadales']


In [47]:
# Splitting the human dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train_o, X_test_o, y_train_o, y_test_o = train_test_split(X, 
                                                    labels_orders, 
                                                    test_size = 0.10, 
                                                    random_state=42)

In [48]:
print(X_train_o.shape)
print(X_test_o.shape)

(25401, 272193)
(2823, 272193)


In [49]:
### Multinomial Naive Bayes Classifier ###
# The alpha parameter was determined by grid search previously
from sklearn.naive_bayes import MultinomialNB
classifier_o = MultinomialNB(alpha=0.1)
classifier_o.fit(X_train_o, y_train_o)

MultinomialNB(alpha=0.1)

In [50]:
y_pred_o = classifier_o.predict(X_test_o)
y_pred_o

array([19, 13, 13, ..., 13,  5, 13], dtype=int64)

In [51]:
print("Confusion matrix\n")
print(pd.crosstab(pd.Series(y_test_o, name='Actual'), pd.Series(y_pred_o, name='Predicted')))
def get_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1 = get_metrics(y_test_o, y_pred_o)
print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))

Confusion matrix

Predicted  0    1    2   3   4   5   6   7    8    9    10  11   12   13  14  \
Actual                                                                         
0          20    0    0   0   0   0   0   0    0    0    0   0    0    0   0   
1           3  306    0   1   0   2   0   0    0   11    0   8    2    0   0   
2           0    0  108   0   0   0   0   0    0    0    0   0    3    3   0   
3           0    0    0  48   0   0   0   0    0    0    0   0    0    0   0   
4           0    0    0   0  37   0   0   0    0    0    0   0    0    0   0   
5           0    4    0   0   0  23   1   0    0    1    0   0    0    0   0   
6           0    0    0   0   0   0  57   0    0    0    0   0    0    0   0   
7           0    0    0   0   0   0   0  19    1    0    0   0    0    0   0   
8           0    0    0   0   0   0   0   0  138    0    0   0    0    0   0   
9           1   12    0   0   1   1   0   0    0  306    0   2    1    0   0   
10          0    1    

In [52]:
def predict_o(seq):
    if type(seq)== str:
        seq = [seq]
    return classifier_o.predict(cv.transform(seq))

def find_order(seq):
    pred = predict_o(seq)
    return [orders[i] for i in pred]

In [53]:
predict_o(human_texts[3]).item()

13

In [54]:
find_order(human_texts[0:5])

['order_Alteromonadales',
 'order_Rhodobacterales',
 'order_Vibrionales',
 'order_Rhodobacterales',
 'order_Alteromonadales']

# Family

In [55]:
print(list(sg.groupby('Family').count().index),'\n', len(sg.groupby('Family').count().index))

['Aeromonadaceae', 'Alteromonadaceae', 'Alteromonadales_incertae_sedis', 'Bradyrhizobiaceae', 'Brucellaceae', 'Caulobacteraceae', 'Chromatiaceae', 'Colwelliaceae', 'Coxiellaceae', 'Erythrobacteraceae', 'Ferrimonadaceae', 'Ferrovaceae', 'Hahellaceae', 'Halomonadaceae', 'Hyphomicrobiaceae', 'Hyphomonadaceae', 'Idiomarinaceae', 'Legionellaceae', 'Methylophilaceae', 'Moritellaceae', 'Neisseriaceae', 'Oceanospirillaceae', 'Oceanospirillales_incertae_sedis', 'Pasteurellaceae', 'Phyllobacteriaceae', 'Piscirickettsiaceae', 'Pseudoalteromonadaceae', 'Pseudomonadaceae', 'Psychromonadaceae', 'Rhizobiaceae', 'Rhodobacteraceae', 'Rhodospirillaceae', 'Simiduia', 'Sinobacteraceae', 'Sphingomonadaceae', 'Vibrionaceae', 'Xanthobacteraceae', 'Xanthomonadaceae', '\\"Salinisphaeraceae\\"'] 
 39


In [56]:
families = []
for i in sg['Family']:
    if i[0:2]==   r"\"":
        i = i[2:-2]
    families.append(i)

In [57]:
sg['Family'] = families

In [58]:
print(list(sg.groupby('Family').count().index), len(sg.groupby('Family').count().index))

['Aeromonadaceae', 'Alteromonadaceae', 'Alteromonadales_incertae_sedis', 'Bradyrhizobiaceae', 'Brucellaceae', 'Caulobacteraceae', 'Chromatiaceae', 'Colwelliaceae', 'Coxiellaceae', 'Erythrobacteraceae', 'Ferrimonadaceae', 'Ferrovaceae', 'Hahellaceae', 'Halomonadaceae', 'Hyphomicrobiaceae', 'Hyphomonadaceae', 'Idiomarinaceae', 'Legionellaceae', 'Methylophilaceae', 'Moritellaceae', 'Neisseriaceae', 'Oceanospirillaceae', 'Oceanospirillales_incertae_sedis', 'Pasteurellaceae', 'Phyllobacteriaceae', 'Piscirickettsiaceae', 'Pseudoalteromonadaceae', 'Pseudomonadaceae', 'Psychromonadaceae', 'Rhizobiaceae', 'Rhodobacteraceae', 'Rhodospirillaceae', 'Salinisphaeraceae', 'Simiduia', 'Sinobacteraceae', 'Sphingomonadaceae', 'Vibrionaceae', 'Xanthobacteraceae', 'Xanthomonadaceae'] 39


In [59]:
families = list(pd.get_dummies(sg['Family']).columns)
families = ['familiy_' + i for i in families]

labels_families = np.argmax(np.array(pd.get_dummies(sg['Family'])), axis=1)
print(labels_families, len(labels_families))

[19 30 36 ... 30 37 38] 28224


In [60]:
print(families)

['familiy_Aeromonadaceae', 'familiy_Alteromonadaceae', 'familiy_Alteromonadales_incertae_sedis', 'familiy_Bradyrhizobiaceae', 'familiy_Brucellaceae', 'familiy_Caulobacteraceae', 'familiy_Chromatiaceae', 'familiy_Colwelliaceae', 'familiy_Coxiellaceae', 'familiy_Erythrobacteraceae', 'familiy_Ferrimonadaceae', 'familiy_Ferrovaceae', 'familiy_Hahellaceae', 'familiy_Halomonadaceae', 'familiy_Hyphomicrobiaceae', 'familiy_Hyphomonadaceae', 'familiy_Idiomarinaceae', 'familiy_Legionellaceae', 'familiy_Methylophilaceae', 'familiy_Moritellaceae', 'familiy_Neisseriaceae', 'familiy_Oceanospirillaceae', 'familiy_Oceanospirillales_incertae_sedis', 'familiy_Pasteurellaceae', 'familiy_Phyllobacteriaceae', 'familiy_Piscirickettsiaceae', 'familiy_Pseudoalteromonadaceae', 'familiy_Pseudomonadaceae', 'familiy_Psychromonadaceae', 'familiy_Rhizobiaceae', 'familiy_Rhodobacteraceae', 'familiy_Rhodospirillaceae', 'familiy_Salinisphaeraceae', 'familiy_Simiduia', 'familiy_Sinobacteraceae', 'familiy_Sphingomonadac

In [61]:
len(families)

39

In [62]:
# Splitting the human dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(X, 
                                                    labels_families, 
                                                    test_size = 0.10, 
                                                    random_state=42)

In [63]:
print(X_train_f.shape)
print(X_test_f.shape)

(25401, 272193)
(2823, 272193)


In [64]:
### Multinomial Naive Bayes Classifier ###
# The alpha parameter was determined by grid search previously
from sklearn.naive_bayes import MultinomialNB
classifier_f = MultinomialNB(alpha=0.1)
classifier_f.fit(X_train_f, y_train_f)

MultinomialNB(alpha=0.1)

In [65]:
y_pred_f = classifier_f.predict(X_test_f)
y_pred_f

array([38, 30, 30, ..., 30, 33, 30], dtype=int64)

In [66]:
print("Confusion matrix\n")
print(pd.crosstab(pd.Series(y_test_f, name='Actual'), pd.Series(y_pred_f, name='Predicted')))
def get_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1 = get_metrics(y_test_f, y_pred_f)
print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))

Confusion matrix

Predicted  0    1   2   3   4   5   6   7   8   9   ...  29   30  31  32  33  \
Actual                                              ...                        
0          20    0   0   0   0   0   0   0   0   0  ...   0    0   0   0   0   
1           0  106   1   0   0   0   0   0   0   0  ...   0    0   0   1   1   
2           0    0  29   0   0   0   0   0   0   0  ...   0    0   0   0   1   
3           0    0   0  76   0   0   0   0   0   0  ...   0    0   0   0   0   
4           0    0   0   0  28   0   0   0   0   0  ...   0    0   0   0   0   
5           0    0   0   0   0  61   0   0   0   0  ...   0    0   0   0   0   
6           0    0   0   0   0   0  48   0   0   0  ...   0    0   0   0   0   
7           0    0   0   0   0   0   0  28   0   0  ...   0    0   0   0   0   
8           0    0   0   0   0   0   0   0  30   0  ...   0    0   0   0   0   
9           0    0   0   0   0   0   0   0   0  52  ...   0    0   0   0   0   
10          0    0   0

In [67]:
def predict_f(seq):
    if type(seq)== str:
        seq = [seq]
    return classifier_f.predict(cv.transform(seq))

def find_family(seq):
    pred = predict_f(seq)
    return [families[i] for i in pred]

In [68]:
predict_f(human_texts[3]).item()

30

In [69]:
find_family(human_texts[0:5])

['familiy_Moritellaceae',
 'familiy_Rhodobacteraceae',
 'familiy_Vibrionaceae',
 'familiy_Rhodobacteraceae',
 'familiy_Ferrimonadaceae']

# GENUS

In [70]:
print(list(sg.groupby('Genus').count().index),'\n', len(sg.groupby('Genus').count().index))

['Actinobacillus', 'Afipia', 'Agarivorans', 'Aggregatibacter', 'Algimonas', 'Aliidiomarina', 'Aliiroseovarius', 'Allochromatium', 'Alteromonas', 'Amphritea', 'Amylibacter', 'Ancylobacter', 'Aquaspirillum', 'Azorhizobium', 'Brevundimonas', 'Caulobacter', 'Celeribacter', 'Cellvibrio', 'Chelonobacter', 'Chitinibacter', 'Chromohalobacter', 'Cobetia', 'Coxiella', 'Croceicoccus', 'Erythrobacter', 'Falsochrobactrum', 'Ferrimonas', 'Ferrovum', 'Hahella', 'Halovibrio', 'Henriciella', 'Histophilus', 'Kingella', 'Labrys', 'Legionella', 'Leisingera', 'Loktanella', 'Lucibacterium', 'Luteibacter', 'Marinobacter', 'Marinomonas', 'Marinospirillum', 'Maritalea', 'Marivita', 'Mesorhizobium', 'Methylobacillus', 'Moritella', 'Nautella', 'Necropsobacter', 'Neisseria', 'Neorhizobium', 'Nevskia', 'Nicoletella', 'Nitrincola', 'Nitrosococcus', 'Niveispirillum', 'Novispirillum', 'Oceanisphaera', 'Pannonibacter', 'Paraglaciecola', 'Pedomicrobium', 'Pelagibacterium', 'Pelagibius', 'Phaeobacter', 'Phocoenobacter',

In [71]:
genes = list(pd.get_dummies(sg['Genus']).columns)
genes = ['genus_' + i for i in genes]

labels_genes = np.argmax(np.array(pd.get_dummies(sg['Genus'])), axis=1)
print(labels_genes, len(labels_genes))

[46 82 37 ...  6 13 92] 28224


In [72]:
print(genes)
len(genes)

['genus_Actinobacillus', 'genus_Afipia', 'genus_Agarivorans', 'genus_Aggregatibacter', 'genus_Algimonas', 'genus_Aliidiomarina', 'genus_Aliiroseovarius', 'genus_Allochromatium', 'genus_Alteromonas', 'genus_Amphritea', 'genus_Amylibacter', 'genus_Ancylobacter', 'genus_Aquaspirillum', 'genus_Azorhizobium', 'genus_Brevundimonas', 'genus_Caulobacter', 'genus_Celeribacter', 'genus_Cellvibrio', 'genus_Chelonobacter', 'genus_Chitinibacter', 'genus_Chromohalobacter', 'genus_Cobetia', 'genus_Coxiella', 'genus_Croceicoccus', 'genus_Erythrobacter', 'genus_Falsochrobactrum', 'genus_Ferrimonas', 'genus_Ferrovum', 'genus_Hahella', 'genus_Halovibrio', 'genus_Henriciella', 'genus_Histophilus', 'genus_Kingella', 'genus_Labrys', 'genus_Legionella', 'genus_Leisingera', 'genus_Loktanella', 'genus_Lucibacterium', 'genus_Luteibacter', 'genus_Marinobacter', 'genus_Marinomonas', 'genus_Marinospirillum', 'genus_Maritalea', 'genus_Marivita', 'genus_Mesorhizobium', 'genus_Methylobacillus', 'genus_Moritella', 'ge

100

In [73]:
# Splitting the human dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(X, 
                                                    labels_genes, 
                                                    test_size = 0.10, 
                                                    random_state=42)

In [74]:
print(X_train_g.shape)
print(X_test_g.shape)

(25401, 272193)
(2823, 272193)


In [75]:
### Multinomial Naive Bayes Classifier ###
# The alpha parameter was determined by grid search previously
from sklearn.naive_bayes import MultinomialNB
classifier_g = MultinomialNB(alpha=0.1)
classifier_g.fit(X_train_g, y_train_g)

MultinomialNB(alpha=0.1)

In [76]:
y_pred_g = classifier_g.predict(X_test_g)
y_pred_g

array([96, 67, 10, ..., 47, 87, 69], dtype=int64)

In [77]:
print("Confusion matrix\n")
print(pd.crosstab(pd.Series(y_test_g, name='Actual'), pd.Series(y_pred_g, name='Predicted')))
def get_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1 = get_metrics(y_test_g, y_pred_g)
print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))

Confusion matrix

Predicted  0   1   2   3   4   5   6   7   8   9   ...  90  91  92  93  94  \
Actual                                             ...                       
0          15   0   0   2   0   0   0   0   0   0  ...   0   0   0   0   0   
1           0  14   0   0   0   0   0   0   0   0  ...   0   0   0   1   0   
2           0   0  26   0   0   0   0   0   0   0  ...   0   0   0   0   0   
3           0   0   0  36   0   0   0   0   0   0  ...   0   0   0   0   0   
4           0   0   0   0  29   0   0   0   0   0  ...   0   0   0   0   0   
...        ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..  ..   
95          0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   
96          0   0   0   0   0   0   0   0   0   0  ...   0   0   4   0   0   
97          0   0   0   0   0   0   0   0   1   0  ...   0   0   0   0   0   
98          0   0   0   0   0   0   0   0   0   0  ...   0   2   0   0   0   
99          0   0   0   0   0   0   0   0   0 

In [78]:
def predict_g(seq):
    if type(seq)== str:
        seq = [seq]
    return classifier_g.predict(cv.transform(seq))

def find_genus(seq):
    pred = predict_g(seq)
    return [genes[i] for i in pred]

In [79]:
predict_g(human_texts[3]).item()

77

In [80]:
find_genus(human_texts[0:5])

['genus_Moritella',
 'genus_Sagittula',
 'genus_Lucibacterium',
 'genus_Rhodobaca',
 'genus_Ferrimonas']

# Test

In [81]:
def predict(seq):
    if type(seq) != list:
        seq = [seq]

    t = pd.DataFrame()
    t['sequence'] = [str(i) for i in seq]
    t_texts = list(t.apply(lambda x: getKmers(x['sequence']), axis=1))
    for item in range(len(t_texts)):
        t_texts[item] = ' '.join(t_texts[item])
    
    class_ = find_class(t_texts)
    order = find_order(t_texts)
    family = find_family(t_texts)
    genus = find_genus(t_texts)
    
    t['Class'] = class_
    t['Order'] = order
    t['Family'] = family
    t['Genus'] = genus
    return t

In [82]:
predict(sg_reads_seq[0:5])

,sequence,Class,Order,Family,Genus
0,GCCCCGGGCTCAACCTGGGAACTGCATTTCGAACTGGCAAACTAGA...,class_Gammaproteobacteria,order_Alteromonadales,familiy_Moritellaceae,genus_Moritella
1,ACTGCCTCCAAAACTATCAGTCTAGAGTTCGAGAGAGGTGAGTGGA...,class_Alphaproteobacteria,order_Rhodobacterales,familiy_Rhodobacteraceae,genus_Sagittula
2,TAGCGGCGTCGTTTGACGTTAGCGACAGAAGAAGCACCGGCTAACT...,class_Gammaproteobacteria,order_Vibrionales,familiy_Vibrionaceae,genus_Lucibacterium
3,CTAGCGTTGTTCGGAATTACTGGGCGTATAGCGCGCGTTAGGCGGA...,class_Alphaproteobacteria,order_Rhodobacterales,familiy_Rhodobacteraceae,genus_Rhodobaca
4,CATGGCTCAGATTGAACGCTGGCGGCAGGCCTAACACATGCAAGTC...,class_Gammaproteobacteria,order_Alteromonadales,familiy_Ferrimonadaceae,genus_Ferrimonas


In [83]:
pd.DataFrame(sg_reads_tax[0:5])

,0,1,2,3
0,Gammaproteobacteria,Alteromonadales,Moritellaceae,Moritella
1,Alphaproteobacteria,Rhodobacterales,Rhodobacteraceae,Sagittula
2,Gammaproteobacteria,"\""Vibrionales\""",Vibrionaceae,Lucibacterium
3,Alphaproteobacteria,Rhodobacterales,Rhodobacteraceae,Rhodobaca
4,Gammaproteobacteria,Alteromonadales,Ferrimonadaceae,Ferrimonas


In [86]:
print(sg_reads_seq[0])

GCCCCGGGCTCAACCTGGGAACTGCATTTCGAACTGGCAAACTAGAGTTCTTGAGAGGGTGGTAGAATTTCAGGTGTAGCGGTGAAATGCGAAGAGATCTGAAGGATTACCAGTGGCGAAGGCGGCCTACCGGCAACTATCCGACTCTCGGATGCGAAAGCGAGGTACCGGACGGGATCCGTTGAGCTGTATTTGCCCGCGGTAAATGGATTCCGCTAGCGGGACTCGGAGAGCCTTAGTGGAACCAC
